In [10]:
import sqlite3
import pandas as pd
import numpy as np

# <span style='color:teal'> [문제 1] </span>
투수들의 기록중에서 평균자책점(ERA), 투구인닝(IP), 탈삼진(SO) 기록을 찾아서 Pitcher_stats 란 테이블을 만들고, Eagles 테이블과 Join 하여백넘버, 선수명, 포지션, 투구인닝, 평균자책점, 탈삼진 필드를 갖는 데이터 프레임을 만들어서Join 한 결과를 입력하고, 그 결과를 보이시오.

In [11]:
conn = sqlite3.connect('./test.db') 

### **1. Eagles 테이블 만들기**

In [16]:
# 1  =============================================================================
cur = conn.cursor() 
cur.execute('CREATE TABLE IF NOT EXISTS Eagles \
    (back_no INT NOT NULL, \
     name TEXT, \
     position TEXT, \
     hands TEXT, \
     highschool TEXT, \
     height INT, \
     PRIMARY KEY(back_no));')

In [17]:
cur = conn.cursor()
cur.execute("INSERT INTO Eagles VALUES \
    (1, '하주석', '내야수', '우투좌타', '신일고', 184), \
    (28, '양성우', '외야수', '우투좌타', '충암고', 177);")

In [18]:
conn.commit()

In [19]:
players = pd.read_csv('./players.csv', encoding='EUC-KR')
players

back_no name position hands highschool  height
0       17  김범수       투수  좌투좌타      천안북일고     181
1       38  안영명       투수  우투우타      천안북일고     183
2       36  장민재       투수  우투우타      광주제일고     184
3       13  최재훈       포수  우투우타        덕수고     178
4       52  김태균      내야수  우투우타      천안북일고     185
5        7  송광민      내야수  우투우타        공주고     184
6       50  이성열      내야수  우투좌타       순천효고     185
7       43  정은원      내야수  우투좌타        인천고     177
8        8  정근우      외야수  우투우타        부산고     172
9       30   호잉      외야수  우투좌타        외국고     190

In [20]:
# 4  =============================================================================
cur = conn.cursor()
sql = 'INSERT INTO Eagles VALUES (?, ?, ?, ?, ?, ?);'
# 밑에 sql의 자리에 그대로 넣어도 되지만 보통은 이렇게 따로 만들어서 해놓는다.
# 프로그램에서 물음표 값을 채워넣는다.

for i in range(10):
    cur.execute(sql, (int(players.iloc[i,0]), 
                      players.iloc[i,1], players.iloc[i,2], 
                      players.iloc[i,3], players.iloc[i,4], 
                      int(players.iloc[i,5])))
conn.commit()

cur = conn.cursor()
cur.execute('SELECT * FROM Eagles')

for row in cur:
    print(row)

(1, '하주석', '내야수', '우투좌타', '신일고', 184)
(28, '양성우', '외야수', '우투좌타', '충암고', 177)
(17, '김범수', '투수', '좌투좌타', '천안북일고', 181)
(38, '안영명', '투수', '우투우타', '천안북일고', 183)
(36, '장민재', '투수', '우투우타', '광주제일고', 184)
(13, '최재훈', '포수', '우투우타', '덕수고', 178)
(52, '김태균', '내야수', '우투우타', '천안북일고', 185)
(7, '송광민', '내야수', '우투우타', '공주고', 184)
(50, '이성열', '내야수', '우투좌타', '순천효고', 185)
(43, '정은원', '내야수', '우투좌타', '인천고', 177)
(8, '정근우', '외야수', '우투우타', '부산고', 172)
(30, '호잉', '외야수', '우투좌타', '외국고', 190)


### **2. Pitcher 테이블 만들기**

In [49]:
# 1  =============================================================================
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS Pitcher \
    (id INT NOT NULL, \
     player TEXT, \
     ERA REAL, \
     IP INT, \
     SO INT, \
     PRIMARY KEY(id));')

# 2  =============================================================================
Pitcher = pd.read_csv('./Pitcher.csv',encoding='EUC-KR')
Pitcher

player  \tERA  \tIP  \tSO
0    김범수   5.67    81    65
1    안영명   3.43    44    35
2    장민재   4.81    91    84

In [50]:
# 3  =============================================================================
cur = conn.cursor()
sql = 'INSERT INTO Pitcher VALUES (?, ?, ?, ?, ?);'

for i in range(3):
    cur.execute(sql, (i+1,
                      Pitcher.iloc[i,0],
                      float(Pitcher.iloc[i,1]),
                      int(Pitcher.iloc[i,2]),
                      int(Pitcher.iloc[i,3])))
conn.commit()
 
# 4  =============================================================================
cur = conn.cursor()
cur.execute('SELECT * FROM Pitcher')

for row in cur:
    print(row)

(1, '김범수', 5.67, 81, 65)
(2, '안영명', 3.43, 44, 35)
(3, '장민재', 4.81, 91, 84)


### **3. join하기**

In [46]:
# 1. =============================================================================
sql = "SELECT e.back_no, e.name, e.position, \
        p.IP, p.ERA, p.SO \
        FROM Eagles AS e JOIN Pitcher AS p \
        ON e.name like p.player;"
    
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();


# 7.  =============================================================================
columnName = ['등번호','선수명','포지션','투구인닝','평균자책점','탈삼진']
eagles_df = pd.DataFrame(columns = columnName) 

for row in rows: #row는 tuple 하나씩이다
    eagles_df = eagles_df.append(pd.DataFrame( [list(row)], columns = columnName ),
                                 ignore_index=True)
eagles_df

등번호  선수명 포지션 투구인닝  평균자책점 탈삼진
0  17  김범수  투수   81   5.67  65
1  38  안영명  투수   44   3.43  35
2  36  장민재  투수   91   4.81  84

# <span style='color:teal'> [문제 2] </span>

### 1. girl_group 테이블 만들기
- 국내의 대표적인 걸그룹 또는 보이그룹 5개 이상에 대하여 
- id(*), group_name, 구성원 수, 데뷔일자, 소속사를 갖는 테이블을 만드시오.(*는 Primary Key)

In [87]:
# 1  =============================================================================
cur = conn.cursor() 
cur.execute('CREATE TABLE IF NOT EXISTS girl_group \
    (id INT NOT NULL, \
     group_name TEXT, \
     member INT, \
     debut TEXT, \
     agency TEXT, \
     PRIMARY KEY(id));')

# 2  =============================================================================
girl_group = pd.read_csv('./girl_group.csv',encoding='EUC-KR')
print(girl_group)
print()
# 3  =============================================================================
cur = conn.cursor()
sql = 'INSERT INTO girl_group VALUES (?, ?, ?, ?, ?);'

for i in range(5):
    cur.execute(sql, (i+1,
                      girl_group.iloc[i,0],
                      int(girl_group.iloc[i,1]),
                      girl_group.iloc[i,2],
                      girl_group.iloc[i,3]))
conn.commit()
 
# 4  =============================================================================
cur = conn.cursor()
cur.execute('SELECT * FROM girl_group')

for row in cur:
    print(row)

  group_name  \tmember     \tdebut \tagency
0      TWICE         9  2015-10-20      JYP
1  RedVelvet         5  2014-08-01       SM
2  BLACKPINK         4  2016-08-08       YG
3       ITZY         5  2019-02-12      JYP
4    MAMAMOO         4  2014-06-19      RBW

(1, 'TWICE', 9, '2015-10-20', 'JYP')
(2, 'RedVelvet', 5, '2014-08-01', 'SM')
(3, 'BLACKPINK', 4, '2016-08-08', 'YG')
(4, 'ITZY', 5, '2019-02-12', 'JYP')
(5, 'MAMAMOO', 4, '2014-06-19', 'RBW')


### 2. song 테이블 만들기
- 이들이 불렀던 노래 또는 다른 사람이 불렀던 노래 10곡 이상에 대하여 다음의 정보를 갖는 테이블을 만드시오.  

- song_id(*), song_name, 그룹 id, 발표년도, 작곡가, 도입부 가사 

In [88]:
# 1  =============================================================================
cur = conn.cursor() 
cur.execute('CREATE TABLE IF NOT EXISTS song \
    (song_id INT NOT NULL, \
     song_name TEXT, \
     group_id INT, \
     Release TEXT, \
     composer TEXT, \
     lyrics TEXT, \
     PRIMARY KEY(song_id));')

# 2  =============================================================================
song = pd.read_csv('./song.csv',encoding='EUC-KR')
print(song)
print()
# 3  =============================================================================
cur = conn.cursor()
sql = 'INSERT INTO song VALUES (?, ?, ?, ?, ?, ?);'

for i in range(10):
    cur.execute(sql, (i+1,
                      song.iloc[i,0],
                      int(song.iloc[i,1]),
                      song.iloc[i,2],
                      song.iloc[i,3],
                      song.iloc[i,4]))
conn.commit()
 
# 4  =============================================================================
cur = conn.cursor()
cur.execute('SELECT * FROM song')

for row in cur:
    print(row)

              song_name  group_id     Release              composer  \
0             OOH-AHH하게         1  2015-10-20               블랙아이드필승   
1  Dance The Night Away         1  2018-07-09       Anne Judith Wik   
2                Rookie         2  2017-02-01  Jamil `Digi` Chammas   
3                  짐살라빔         2  2019-06-19         Olof Lindskog   
4                   붐바야         3  2016-08-08                 TEDDY   
5        Kill This Love         3  2019-04-05                 TEDDY   
6                 달라달라?         4  2019-02-12                 별들의전쟁   
7                   ICY         4  2019-07-29                   박진영   
8               Mr.애매모호         5  2014-06-18                   김도훈   
9                  고고베베         5  2019-03-14                   김건모   

                                lyrics  
0                 모두 나를 가지고 매일 가만 안 두죠  
1                오늘이 마지막인 듯 소리 질러 저 멀리  
2                    당당히 넌 고개를 들고 나를 봐  
3            Zimzalabim Zim-zimzalabim  
4            

### 3. girl_group & song 테이블 join
- 위 두개의 테이블을 조인한 결과를 가지고 다음의 필드를 갖는 데이터 프레임을 만드시오.  

- 그룹 이름, 구성원 수, 데뷔 일자, 노래 이름, 발표 년도

In [89]:
# 1. =============================================================================
sql = "SELECT g.group_name, g.member, g.debut, \
        s.song_name, s.Release \
        FROM girl_group AS g JOIN song AS s \
        ON g.id like s.group_id;"
    
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();


# 7.  =============================================================================
columnName = ['그룹이름','구성원 수','데뷔 일자','노래 이름','발표 년도']
girl_group_df = pd.DataFrame(columns = columnName) 

for row in rows: #row는 tuple 하나씩이다
    girl_group_df = girl_group_df.append(pd.DataFrame( [list(row)], columns = columnName ),
                                 ignore_index=True)
girl_group_df

그룹이름 구성원 수       데뷔 일자                 노래 이름       발표 년도
0      TWICE     9  2015-10-20             OOH-AHH하게  2015-10-20
1      TWICE     9  2015-10-20  Dance The Night Away  2018-07-09
2  RedVelvet     5  2014-08-01                Rookie  2017-02-01
3  RedVelvet     5  2014-08-01                  짐살라빔  2019-06-19
4  BLACKPINK     4  2016-08-08                   붐바야  2016-08-08
5  BLACKPINK     4  2016-08-08        Kill This Love  2019-04-05
6       ITZY     5  2019-02-12                 달라달라?  2019-02-12
7       ITZY     5  2019-02-12                   ICY  2019-07-29
8    MAMAMOO     4  2014-06-19               Mr.애매모호  2014-06-18
9    MAMAMOO     4  2014-06-19                  고고베베  2019-03-14

# <span style='color:teal'> [문제 3] </span>

사용자의 이름과, 비밀번호를 갖는 Users 테이블이 있다.  
사용자의 이름과 비밀번호를 올바르게 입력하면 ‘성공’을 출력하고,  
잘못 입력하면 ‘실패’를 출력하는 프로그램을 작성하시오.


In [15]:
# 1  =============================================================================
cur = conn.cursor() 
cur.execute('CREATE TABLE IF NOT EXISTS Rog_in \
    (MYid TEXT NOT NULL, \
     PASSWORD INT, \
     PRIMARY KEY(MYid));')

# 2  =============================================================================
Rog_in = pd.read_csv('./Rog_in.csv',encoding='EUC-KR')
print(Rog_in)
print()

# 3  =============================================================================
cur = conn.cursor()
sql = 'INSERT INTO Rog_in VALUES (?, ?);'

for i in range(6):
    cur.execute(sql, (Rog_in.iloc[i,0],
                      int(Rog_in.iloc[i,1])))
conn.commit()
 
# 4  =============================================================================
cur = conn.cursor()
cur.execute('SELECT * FROM Rog_in')

columnName = ['ID','PASSWORD']

Rog_in_df = pd.DataFrame(columns = columnName) 
for row in cur:
    Rog_in_df = Rog_in_df.append(pd.DataFrame([list(row)],columns = columnName),
                                 ignore_index=True) 
    
Rog_in_df

   Myid  PASSWORD
0  abcd      1234
1  efgh      5678
2  ijkl     12345
3  mnop     56789
4  qrst       123
5  uvwx       456



ID PASSWORD
0  abcd     1234
1  efgh     5678
2  ijkl    12345
3  mnop    56789
4  qrst      123
5  uvwx      456

In [52]:
def login(ID,PASSWORD):
    for i in range(0,6):
        if Rog_in_df['ID'][i] == ID and Rog_in_df['PASSWORD'][i] == PASSWORD:
            return 'Log_in 되었습니다'
        elif Rog_in_df['ID'][i] != ID and Rog_in_df['PASSWORD'][i] == PASSWORD:
            return '아이디가 틀립니다.'
        elif Rog_in_df['ID'][i] == ID and Rog_in_df['PASSWORD'][i] != PASSWORD:
            return '비밀번호가 틀립니다.'
    else:
        return '실패하였습니다.'

In [53]:
ID = input('ID를 입력해주세요> ')
PASSWORD = int(input('비밀번호를 입력해주세요> '))

login(ID,PASSWORD)

ID를 입력해주세요> abcd
비밀번호를 입력해주세요> 1234


'Log_in 되었습니다'

- [비밀번호 프로그램 참고](https://gist.github.com/studybee)

# 테이블 삭제

In [14]:
cur.execute("DROP TABLE Rog_in") # SQLite에 있는 파일명 입력
conn.commit()